In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from preprop import *

In [5]:
df = pd.read_csv('samples/pet.label.csv')
df.shape

(17458, 10)

In [6]:
df.head()

,보호자명,환자명,품종코드,생일,성별코드,검사일,병원,주진단코드,진단코드,SE
0,김소야*7,코코,91.0,2001-05-03 00:00:00,MN,2018-07-04 10:51:01.657,D,2078,"2078,만성신질환, 신장결석, 방광염","S) 식욕부진, 기력저하, 눈 O) - 2일동안 식욕이 거의 없어서 보호자분..."
1,이지은*6,힘찬이,119.0,2007-06-04 00:00:00,MN,2018-04-26 11:33:10.467,D,2244,"2244,담낭점액종","S) 다음다뇨, 식욕부진 O) - 최근들어 다음다뇨 증상과 2~3일전부터는 ..."
2,홍성미,비쥬,499.0,2004-01-01 00:00:00,FS,2018-07-20 14:48:07.967,D,2082,"2082,만성신질환, 부비동염","[오전 통화 by 관중] - 일주일동안 밥안먹고, 활력저하됨 - 보호자님 복부촉..."
3,김래경,세순이,107.0,2001-08-01 00:00:00,FS,2017-06-16 14:56:11.267,D,2087,"2087,단백뇨, 쿠싱, 빈혈",1. 쿠싱약 안먹고 와서 쿠싱검사는 28일에 하기로 함 2. 흉방에서 CVC 허탈...
4,김지현*6,효,125.0,2009-11-05 00:00:00,MN,2018-02-28 10:26:20.280,D,2081,"2081,방광결석","S) 건강검진 진행함 O) 1. 혈액검사 - 간수치 상승(ALP, ALT ..."


In [7]:
print(df.shape)
df.dropna(subset=['SE'], inplace=True)
print(df.shape)
# df.dropna(subset=['주진단코드'], inplace=True)
# print(df.shape)

(17458, 10)
(15974, 10)


In [8]:
df['len'] = df.SE.apply(lambda x : len(str(x)))
print(df.shape)
df = df[df['len'] > 50]
df = df[df['len'] < 5000]
print(df.shape)

(15974, 11)
(15267, 11)


In [9]:
df.SE = df.SE.apply(lambda x : preprocess(str(x)))

In [10]:
df['주진단코드'] = df['주진단코드'].apply(lambda x : label_regex(x))

In [60]:
# df['주진단코드'].value_counts()[0:60]

In [61]:
print(df.shape)
df = df[df['주진단코드']!='normal']
print(df.shape)

(13577, 11)
(12224, 11)


In [70]:
df['주진단코드'].value_counts()[20:80].sum()

3713

In [71]:
label_dict = df['주진단코드'].value_counts()[20:80].to_dict()
def make_label_id(label) :
    if not label in label_dict.keys() :
        label = np.nan
    return label

In [72]:
df['label_id'] = ''
df['label_id'] = df['주진단코드'].apply(lambda x : make_label_id(x))
print(df.shape)
df.dropna(subset=['label_id'], inplace=True)
print(df.shape)

(12224, 12)
(3713, 12)


In [11]:
df = df.reset_index()

In [ ]:
diag = pd.read_csv('samples/diag.csv')
diag['질병코드'] = diag['질병코드'].apply(lambda x : str(x).split('.')[0])
diag.head()

In [15]:
# diag['질병코드'].to_list()

In [75]:
df['질병명'] = ''
for i in range(len(df)) :
#     print(i)
#     diag = df_diag[df.loc[i, '진단코드'] == df_diag['질병코드']]['질병명(한글)']
#     print(df.loc[i, '질병명'])
#     print(df.loc[i, '주진단코드'])
    if str(df.loc[i, '주진단코드']).strip() in diag['질병코드'].to_list() :
#         print(df.loc[i, '주진단코드'])
#         print(diag[df.loc[i, '주진단코드'] == diag['질병코드']]['질병명(한글)'].values[0])
        df.loc[i, '질병명'] = diag[df.loc[i, '주진단코드'] == diag['질병코드']]['질병명(한글)'].values[0]
    if (str(df.loc[i, '주진단코드']).strip() == '건강검진') :
        df.loc[i, '질병명'] = '건강검진'
    if (str(df.loc[i, '주진단코드']).strip() == '중성화') :
        df.loc[i, '질병명'] = '중성화'        
#         df.loc[i, '분류'] = df_diag[df.loc[i, '진단코드'] == df_diag['질병코드']]['분류'].values[0]

In [28]:
# df.to_csv('test.csv', index=None)

In [76]:
df = df[['질병명', '주진단코드', 'SE']]

In [77]:
df['질병명'].value_counts().sum()

3713

In [78]:
diag_dict ={}
for i in range(len(df)) :
    diag_dict[df.loc[i, '주진단코드']] = df.loc[i, '질병명']

In [80]:
# diag_dict

In [82]:
df['진단코드'] = ''
for i in range(len(df)) :
#     print(str(df.loc[i, '주진단코드']).strip())
    if str(df.loc[i, '주진단코드']).strip() == '2092' or str(df.loc[i, '주진단코드']).strip() == '2091':
        df.loc[i, '진단코드'] = '방광질환'
    if str(df.loc[i, '주진단코드']).strip() == '2031' or str(df.loc[i, '주진단코드']).strip() == '2022':
        df.loc[i, '진단코드'] = '폐질환'      
    if str(df.loc[i, '주진단코드']).strip() == '2001' or str(df.loc[i, '주진단코드']).strip() == '2002':
        df.loc[i, '진단코드'] = '심장질환'    
    if str(df.loc[i, '주진단코드']).strip() == '2048' or str(df.loc[i, '주진단코드']).strip() == '2043':
        df.loc[i, '진단코드'] = '위장염'   
    if str(df.loc[i, '주진단코드']).strip() == '2084' or str(df.loc[i, '주진단코드']).strip() == '2081':
        df.loc[i, '진단코드'] = '간질환'           
    if str(df.loc[i, '질병명']).strip() == '중성화':
        df.loc[i, '진단코드'] = '중성화'       
    if str(df.loc[i, '질병명']).strip() == '건강검진': 
        df.loc[i, '진단코드'] = '건강검진'      
    if str(df.loc[i, '주진단코드']).strip() == '2046': 
        df.loc[i, '진단코드'] = '소화관 이물' 
    if str(df.loc[i, '주진단코드']).strip() == '2082': 
        df.loc[i, '진단코드'] = '췌장염'  
    if str(df.loc[i, '주진단코드']).strip() == '2087': 
        df.loc[i, '진단코드'] = '신장질환'  
    if str(df.loc[i, '주진단코드']).strip() == '2101': 
        df.loc[i, '진단코드'] = '자궁 축농증'    
    if str(df.loc[i, '주진단코드']).strip() == '2071': 
        df.loc[i, '진단코드'] = '구토설사혈변'    
    if str(df.loc[i, '주진단코드']).strip() == '2170': 
        df.loc[i, '진단코드'] = '치주염'    
    if str(df.loc[i, '주진단코드']).strip() == '2116': 
        df.loc[i, '진단코드'] = '유선종양'    
    if str(df.loc[i, '주진단코드']).strip() == '2095': 
        df.loc[i, '진단코드'] = '고양이하부요로질환'      
    if str(df.loc[i, '주진단코드']).strip() == '2126': 
        df.loc[i, '진단코드'] = '경련 발작'    
    if str(df.loc[i, '주진단코드']).strip() == '2185': 
        df.loc[i, '진단코드'] = '슬개골탈구'        
    if str(df.loc[i, '주진단코드']).strip() == '2274': 
        df.loc[i, '진단코드'] = '교상'      
    if str(df.loc[i, '주진단코드']).strip() == '2244': 
        df.loc[i, '진단코드'] = '당뇨병'   
    if str(df.loc[i, '주진단코드']).strip() == '슬개골탈구': 
        df.loc[i, '진단코드'] = '슬개골탈구' 
    if str(df.loc[i, '주진단코드']).strip() == '2257': 
        df.loc[i, '진단코드'] = '인체 약물에 의한 중독'  
    if str(df.loc[i, '주진단코드']).strip() == '2283': 
        df.loc[i, '진단코드'] = '쇼크 증상'   
    if str(df.loc[i, '주진단코드']).strip() == '2263': 
        df.loc[i, '진단코드'] = '개 디스템퍼'  
    if str(df.loc[i, '주진단코드']).strip() == '2049': 
        df.loc[i, '진단코드'] = '개 파보 바이러스'  
    if str(df.loc[i, '주진단코드']).strip() == '2273': 
        df.loc[i, '진단코드'] = '외상 (타박상 / 찰과상 / 타박상)'  
    if str(df.loc[i, '주진단코드']).strip() == '2255': 
        df.loc[i, '진단코드'] = '초콜릿 중독'  
    if str(df.loc[i, '주진단코드']).strip() == '2119': 
        df.loc[i, '진단코드'] = '간질'   
    if str(df.loc[i, '주진단코드']).strip() == '2267': 
        df.loc[i, '진단코드'] = '고양이 복막염'  
    if str(df.loc[i, '주진단코드']).strip() == '2259': 
        df.loc[i, '진단코드'] = '다른 중독 질환'  
    if str(df.loc[i, '주진단코드']).strip() == '2193': 
        df.loc[i, '진단코드'] = '골절'       
        
    if str(df.loc[i, '주진단코드']).strip() == '스케일링': 
        df.loc[i, '진단코드'] = '스케일링'   
    if str(df.loc[i, '주진단코드']).strip() == '2120' or str(df.loc[i, '주진단코드']).strip() == '2121': 
        df.loc[i, '진단코드'] = '수막염 뇌수종'  
    if str(df.loc[i, '주진단코드']).strip() == '2021': 
        df.loc[i, '진단코드'] = '기관 허탈'  
    if str(df.loc[i, '주진단코드']).strip() == '2058': 
        df.loc[i, '진단코드'] = '장질환'     
        
    if str(df.loc[i, '주진단코드']).strip() == '2232': 
        df.loc[i, '진단코드'] = '혈소판 감소증'  
    if str(df.loc[i, '주진단코드']).strip() == '2233': 
        df.loc[i, '진단코드'] = '비장 이상' 
    if str(df.loc[i, '주진단코드']).strip() == '2090': 
        df.loc[i, '진단코드'] = '급성신부전'
    if str(df.loc[i, '주진단코드']).strip() == '2014': 
        df.loc[i, '진단코드'] = '심장사상충'
    if str(df.loc[i, '주진단코드']).strip() == '2230': 
        df.loc[i, '진단코드'] = '빈혈'       
        
    if str(df.loc[i, '주진단코드']).strip() == '2171': 
        df.loc[i, '진단코드'] = '치근 농양'
    if str(df.loc[i, '주진단코드']).strip() == '2178': 
        df.loc[i, '진단코드'] = '구내염'
    if str(df.loc[i, '주진단코드']).strip() == '2181': 
        df.loc[i, '진단코드'] = '추간판탈출증'
    if str(df.loc[i, '주진단코드']).strip() == '2245': 
        df.loc[i, '진단코드'] = '신피질 커싱'
    if str(df.loc[i, '주진단코드']).strip() == '골절': 
        df.loc[i, '진단코드'] = '골절'   
        
    if str(df.loc[i, '주진단코드']).strip() == '2075': 
        df.loc[i, '진단코드'] = '지방간'
    if str(df.loc[i, '주진단코드']).strip() == '2122': 
        df.loc[i, '진단코드'] = '뇌질환'
    if str(df.loc[i, '주진단코드']).strip() == '2094': 
        df.loc[i, '진단코드'] = '결석증'
    if str(df.loc[i, '주진단코드']).strip() == '2061': 
        df.loc[i, '진단코드'] = '항문낭'
    if str(df.loc[i, '주진단코드']).strip() == '2236':  
        df.loc[i, '진단코드'] = '고양이 백혈구 감소증'   
        
    if str(df.loc[i, '주진단코드']).strip() == '2032': 
        df.loc[i, '진단코드'] = '흉수'
    if str(df.loc[i, '주진단코드']).strip() == '2240': 
        df.loc[i, '진단코드'] = '림프종'
    if str(df.loc[i, '주진단코드']).strip() == '2004': 
        df.loc[i, '진단코드'] = '심근증'
    if str(df.loc[i, '주진단코드']).strip() == '2093': 
        df.loc[i, '진단코드'] = '요도폐색'
    if str(df.loc[i, '주진단코드']).strip() == '2179':  
        df.loc[i, '진단코드'] = '구강내 종양'     
        
    if str(df.loc[i, '주진단코드']).strip() == '2231': 
        df.loc[i, '진단코드'] = '빈혈'
    if str(df.loc[i, '주진단코드']).strip() == '2077':  
        df.loc[i, '진단코드'] = '문맥션트'             

In [83]:
df['진단코드'] = df['진단코드'].apply(lambda x : np.nan if str(x) == '' else x)

In [84]:
print(df.shape)
df.dropna(subset=['진단코드'], inplace=True)
print(df.shape)

(3713, 4)
(2834, 4)


In [94]:
print(len(df['진단코드'].value_counts()))
print(df['진단코드'].value_counts().sum())
print(df['진단코드'].value_counts())

39
2834
폐질환                     221
간질환                     144
교상                      120
빈혈                      118
수막염 뇌수종                 115
당뇨병                     112
추간판탈출증                  104
다른 중독 질환                103
외상 (타박상 / 찰과상 / 타박상)    100
골절                       97
초콜릿 중독                   93
고양이 백혈구 감소증              92
신피질 커싱                   86
고양이 복막염                  82
쇼크 증상                    79
개 파보 바이러스                75
기관 허탈                    68
치근 농양                    67
구내염                      65
스케일링                     60
슬개골탈구                    59
인체 약물에 의한 중독             59
심장사상충                    57
혈소판 감소증                  53
비장 이상                    52
개 디스템퍼                   50
급성신부전                    50
간질                       50
항문낭                      47
장질환                      47
심근증                      46
문맥션트                     36
요도폐색                     36
뇌질환                      33
림프종                      33
지방간         

In [89]:
import pandas as pd
import numpy as np
import os
import re
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [90]:
encoder = LabelEncoder()
encoder.fit(df['진단코드'])
df['label_id'] = encoder.transform(df['진단코드'])

In [92]:
# df.head()

In [93]:
df.to_csv('files/pre-fine.csv', index=None)

In [93]:
train, test = train_test_split(df,test_size=0.1, random_state=100 )
train.to_csv('files/train1.csv', index=None)
test.to_csv('files/test1.csv', index=None)
print(train.shape)
print(test.shape)

(5540, 5)
(616, 5)


In [106]:
train = pd.read_csv('files/train1.csv')
test = pd.read_csv('files/test1.csv')
df = pd.concat([train, test])
df.shape

(6156, 5)

In [107]:
label_dict1 = df['진단코드'].value_counts()[0:10].to_dict()
def make_label_id1(label) :
    if not label in label_dict1.keys() :
        label = np.nan
    return label

In [108]:
label_dict1

{'위장염': 661,
 '심장질환': 607,
 '중성화': 580,
 '소화관 이물': 564,
 '췌장염': 479,
 '신장질환': 479,
 '자궁 축농증': 377,
 '방광질환': 372,
 '구토설사혈변': 324,
 '치주염': 313}

In [109]:
# df['label_id'] = ''
df['진단코드'] = df['진단코드'].apply(lambda x : make_label_id1(x))
print(df.shape)
df.dropna(subset=['진단코드'], inplace=True)
print(df.shape)

(6156, 5)
(4756, 5)


In [110]:
df.head()

,질병명,주진단코드,SE,진단코드,label_id
0,판막증 (의심 포함한 심장 잡음 + 심부전 증후 자),2001,금일 오전 구토 물설사. 스카이에서 폐수종처치를 받음 식욕 활기 백신 사상충 a ...,심장질환,11
1,중성화,중성화,v 식욕 활기 백신 사상충 스켈링 중성화 약갑스 발사 까지 두마리 카드 현금 졸레...,중성화,15
2,만성 신장 질환 (신부전 포함),2087,history 임상증상은 양호합니다. objective 차회 c lymphocyt...,신장질환,10
4,췌장염,2082,모니터링 구토로 내원 내원 시 빈호흡 및 멘탈 상태 좋지 않음 내원 직후 time ...,췌장염,16
5,췌장염,2082,s. 금일 약간 묽은변 아침에 식욕 없었음 o. 이전에 history 있었던 꼬리쪽...,췌장염,16


In [111]:
encoder = LabelEncoder()
encoder.fit(df['진단코드'])
df['label_id'] = encoder.transform(df['진단코드'])

In [112]:
df.head(10)

,질병명,주진단코드,SE,진단코드,label_id
0,판막증 (의심 포함한 심장 잡음 + 심부전 증후 자),2001,금일 오전 구토 물설사. 스카이에서 폐수종처치를 받음 식욕 활기 백신 사상충 a ...,심장질환,4
1,중성화,중성화,v 식욕 활기 백신 사상충 스켈링 중성화 약갑스 발사 까지 두마리 카드 현금 졸레...,중성화,7
2,만성 신장 질환 (신부전 포함),2087,history 임상증상은 양호합니다. objective 차회 c lymphocyt...,신장질환,3
4,췌장염,2082,모니터링 구토로 내원 내원 시 빈호흡 및 멘탈 상태 좋지 않음 내원 직후 time ...,췌장염,8
5,췌장염,2082,s. 금일 약간 묽은변 아침에 식욕 없었음 o. 이전에 history 있었던 꼬리쪽...,췌장염,8
6,만성 신장 질환 (신부전 포함),2087,구토 없으나 점액성 설사를 자주 보임. 스멕타 추가했음. 기저귀를 채웠음. 물은 스...,신장질환,3
7,판막증 (의심 포함한 심장 잡음 + 심부전 증후 자),2001,s 몇일 전부터 심한 건성기침 증상 보임 방사선 전반적인 심비대 후엽 폐패턴 증가 ...,심장질환,4
10,판막증 (의심 포함한 심장 잡음 + 심부전 증후 자),2001,bun 상승 신장보조약제 잘 못먹였다고 함. 잘 먹이고 month 후후 리첵하기로 ...,심장질환,4
11,중성화,중성화,주호소 현증경과 내과 아팠던 적은 없어요. 잘못 먹으면 뱉는 정도 고무줄 먹었을때만...,중성화,7
12,만성 신장 질환 (신부전 포함),2087,refer. 서울종합동물병원 주호소 신장수치 상승 현증경과 이빨이 좋지않아 스켈링...,신장질환,3


In [113]:
train, test = train_test_split(df,test_size=0.1, random_state=100 )
train.to_csv('files/train2.csv', index=None)
test.to_csv('files/test2.csv', index=None)
print(train.shape)
print(test.shape)

(4280, 5)
(476, 5)


In [ ]:
pretrained_path = './pretrained'
model = RobertaForSequenceClassification.from_pretrained(pretrained_path, num_labels=19)

# training is done and save
model.save_pretrained("./finetune3")

pretrained_path = './finetune3'
model = RobertaForSequenceClassification.from_pretrained(pretrained_path, num_labels=10)